In [ ]:
import data_layer_aia as data_layer
import strategy_tools as tool
import joblib
import numpy as np
import pandas as pd
import os

In [ ]:
### Load fund data ###
#output_dict = data_layer.main()
#contents = output_dict['Provider']['MPF Scheme Name']
#df = pd.DataFrame(contents)

with open('fund_data.txt', 'r') as file:
    contents = file.read()
df = pd.read_json(contents)

### Get the dataset for portfolio building ###
df = df.sort_index()
split = int(len(df)*0.8)
test_df = df[split:]

### Construct a df dict and work for features engineering ###
table_dict = {} # dict for storing df
for col in test_df.columns:
    table_dict[col] = pd.DataFrame(test_df[col])

### Initiate the details ###
df['Portfolio_Value'] = 0.0
weight_ls = []
period = 30
available_capital = 100000
portfolio_val = 0.0

for row in test_df.index[period:]:
    cur_row_num = test_df.index.get_loc(row)
    selected_fund = []
    
    ### Loop through each df ###
    for k,v in table_dict.items():
        fund_df = v
        fund_df = tool.get_hsi(fund_df)
        df_w_features = tool.feature_engineer(fund_df)

        ### Retrive the models ###
        current = os.getcwd()
        path = f'{current}/Models'
        filename = f'{path}/{k}.pickle'
        with open(filename, 'rb') as file:
            log_re_model = joblib.load(file)
            svm_model = joblib.load(file)
            lightgbm_model = joblib.load(file)

        ### Use test set to test ###
        if row not in df_w_features.index:
            continue
        y_pred = tool.get_prediction([df_w_features.loc[row][:-1]], [log_re_model, svm_model, lightgbm_model],[0.2,0.3,0.5])
        print(f'For row: {row}, fund {k} s y_pred is {y_pred}')
        if y_pred == 1:
            selected_fund.append(k)
    
    print(f'Total funds: {test_df.columns}')
    print(f'Selected fund in {row}: {selected_fund}')
    print()

    ### Construct portfolio ###
    # Don't have suitable funds to invest -> hold previous postion
    if len(selected_fund) < 1 and row != test_df.index[period]:
        df.loc[row, 'Portfolio_Value'] = portfolio_val
        weight_ls.append(weight_ls[-1])
        continue

    available_capital += portfolio_val # Sell previous position
    train_df = test_df[cur_row_num-(period-1):(cur_row_num+1)]

    ### New position ###
    print('Generate new position')
    # Genarate weights and portfolio information
    portfolio_weights, sharpe_ratio_port = tool.portfolios(train_df)
    max_sharpe_ix = tool.select_maxsharpe(sharpe_ratio_port)
    weights = portfolio_weights[max_sharpe_ix]

    # Capital allocation
    cash_for_buy = weights * available_capital
    fund_purchase_unit = tool.capital_allocation(train_df.loc[row], cash_for_buy)

    # Store the portfolio value
    portfolio_val = np.sum(train_df.loc[row] * fund_purchase_unit)
    df.loc[row, 'Portfolio_Value'] = portfolio_val
    weight_ls.append(weights)

In [ ]:
### performace review ###